In [ ]:
!pip install tf_keras scikit-learn wordcloud tensorflow
# !pip install --upgrade ktrain
!pip install Keras-Preprocessing
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.corpus import brown
from nltk.corpus import gutenberg
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pickle
import joblib
from collections import Counter
from textblob import Word
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Activation, Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten, GRU, Conv1D, MaxPooling1D, Bidirectional
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
import re
sns.set()
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('gutenberg')
nltk.download('brown')
nltk.download("reuters")
nltk.download('words')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 721.5 kB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
df=pd.read_csv("processedData.csv")
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,no,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,yes,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,yes,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,no,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,no,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,no,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,yes,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,yes,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [ ]:
num_species = df['species_group'].nunique()
num_species
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,no,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,yes,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,yes,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,no,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,no,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,no,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,yes,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,yes,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [ ]:
df['tail']=df['tail'].map({'yes':1, 'no':0})
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,0,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,1,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,1,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,1,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,0,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,0,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,1,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_message = vectorizer.fit_transform(df['processedMessage'])

In [ ]:
from scipy.sparse import hstack

X_features = hstack([X_message, np.array(df[['fingers', 'tail']])])
y = df['species']

In [ ]:
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,0,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,1,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,1,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,1,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,0,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,0,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,1,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['species_number'] = label_encoder.fit_transform(df['species_group'])

X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.15, random_state=37)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Function to combine labels
def combine_labels(y):
    return y.replace({
        'Faerix_Group1': 'Faerix',
        'Faerix_Group2': 'Faerix',
        'Emotivor_Group1': 'Emotivor',
        'Emotivor_Group2': 'Emotivor'
    })

# Define the Random Forest model with specified parameters
rfc = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=37, class_weight='balanced')

# Train the model
rfc.fit(X_train, y_train)

# Predict using the trained model
y_pred = rfc.predict(X_val)

# Combine the labels for both predicted and actual values
y_pred4 = combine_labels(pd.Series(y_pred))
y_val_combined = combine_labels(pd.Series(y_val))

# Calculate the accuracy based on the combined labels
accuracy = accuracy_score(y_val_combined, y_pred4)
print(f'Validation Accuracy: {accuracy}')

# Optionally, you can print the classification report as well
print(classification_report(y_val_combined, y_pred4))


Validation Accuracy: 0.88
              precision    recall  f1-score   support

      Aquari       1.00      0.91      0.95        11
       Cybex       0.88      0.78      0.82         9
    Emotivor       0.80      0.89      0.84         9
      Faerix       0.75      1.00      0.86         3
     Florian       0.83      1.00      0.91        10
     Mythron       1.00      0.80      0.89         5
      Nexoon       0.86      0.86      0.86         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.83      0.71      0.77         7
     Zorblax       0.88      1.00      0.93         7

    accuracy                           0.88        75
   macro avg       0.88      0.88      0.88        75
weighted avg       0.89      0.88      0.88        75



## Logistic Regression

In [ ]:
lr = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
lr.fit(X_train, y_train)
y_pred1 = lr.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred1)))
print(classification_report(y_val, y_pred1))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Accuracy: 0.92
              precision    recall  f1-score   support

      Aquari       1.00      0.73      0.84        11
       Cybex       1.00      0.89      0.94         9
    Emotivor       0.89      0.89      0.89         9
      Faerix       1.00      1.00      1.00         3
     Florian       0.77      1.00      0.87        10
     Mythron       1.00      1.00      1.00         5
      Nexoon       1.00      1.00      1.00         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.88      1.00      0.93         7
     Zorblax       0.88      1.00      0.93         7

    accuracy                           0.92        75
   macro avg       0.94      0.94      0.93        75
weighted avg       0.93      0.92      0.92        75



## Support Vector Machines (SVM)

In [ ]:
svc = LinearSVC()
svc.fit(X_train, y_train)
y_pred2 = svc.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred2)))
print(classification_report(y_val, y_pred2))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.84
              precision    recall  f1-score   support

      Aquari       1.00      0.73      0.84        11
       Cybex       1.00      0.89      0.94         9
    Emotivor       0.60      0.67      0.63         9
      Faerix       0.75      1.00      0.86         3
     Florian       0.77      1.00      0.87        10
     Mythron       1.00      0.80      0.89         5
      Nexoon       1.00      1.00      1.00         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.57      0.57      0.57         7
     Zorblax       0.88      1.00      0.93         7

    accuracy                           0.84        75
   macro avg       0.86      0.85      0.85        75
weighted avg       0.86      0.84      0.84        75



/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Naive Bayes

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred3 = mnb.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred3)))
print(classification_report(y_val, y_pred3))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6933333333333334
              precision    recall  f1-score   support

      Aquari       1.00      0.82      0.90        11
       Cybex       1.00      0.56      0.71         9
    Emotivor       1.00      0.11      0.20         9
      Faerix       0.00      0.00      0.00         3
     Florian       1.00      0.60      0.75        10
     Mythron       0.45      1.00      0.62         5
      Nexoon       0.64      1.00      0.78         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.43      0.86      0.57         7
     Zorblax       0.58      1.00      0.74         7

    accuracy                           0.69        75
   macro avg       0.71      0.68      0.62        75
weighted avg       0.80      0.69      0.66        75



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define Gradient Boosting model with valid max_features
gbc = GradientBoostingClassifier(n_estimators=100, max_features='sqrt', max_depth=4, random_state=37, verbose=1)

# Train the model
gbc.fit(X_train, y_train)

# Predict using the trained model
y_pred5 = gbc.predict(X_val)

# Print the accuracy and classification report
print(accuracy_score(y_val, y_pred5))
print(classification_report(y_val, y_pred5))


      Iter       Train Loss   Remaining Time 
         1           2.0686            9.34s
         2           1.8451            9.36s
         3           1.6760            9.14s
         4           1.5424            9.92s
         5           1.4614           10.02s
         6           1.3561           11.16s
         7           1.2759           10.68s
         8           1.1943           10.61s
         9           1.1440           10.56s
        10           1.0751           10.29s
        20           0.6364           10.28s
        30           0.4212            9.09s
        40           0.2915            7.82s
        50           0.2103            6.56s
        60           0.1536            4.68s
        70           0.1147            3.20s
        80           0.0859            1.99s
        90           0.0654            0.94s
       100           0.0496            0.00s
0.76
              precision    recall  f1-score   support

      Aquari       0.83      0.91      

## Adaboost Random Forest

### Hyperparamter Tuning

In [ ]:

from sklearn.model_selection import GridSearchCV

# Define parameter grid for AdaBoost
param_grid_abc = {
    'base_estimator__n_estimators': [50, 100, 150],
    'base_estimator__max_depth': [5, 9, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150]
}

# Create a GridSearchCV object
grid_search_abc = GridSearchCV(abc, param_grid_abc, cv=5, scoring='accuracy')

# Fit the grid search
grid_search_abc.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search_abc.best_params_)
print("Best Score:", grid_search_abc.best_score_)

# Use the best estimator to predict
best_abc = grid_search_abc.best_estimator_
y_pred_best = best_abc.predict(X_val)
print("Best Accuracy: " + str(accuracy_score(y_val, y_pred_best)))
print(classification_report(y_val, y_pred_best))

from sklearn.model_selection import cross_val_score

# Evaluate using cross-validation
scores = cross_val_score(abc, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Scores:", scores)
print("Mean CV Score:", scores.mean())


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=37)
abc= AdaBoostClassifier(base_estimator=rfc, learning_rate=0.05, n_estimators=100)
abc.fit(X_train, y_train)
y_pred7= abc.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val, y_pred7)))
print(classification_report(y_val, y_pred7))

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df[['processedMessage','fingers','tail']],
                                                    df['species_group'])

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2),max_features=128)
vectorizer.fit(df['processedMessage'])

tfidf_X_train = (vectorizer.transform(X_train1['processedMessage'])).toarray()
tfidf_X_test = (vectorizer.transform(X_test1['processedMessage'])).toarray()

label_encoder = LabelEncoder()
label_encoder.fit(df['species_group'])

y_train1 = to_categorical(label_encoder.transform(y_train1),num_classes=12)
y_test1 = to_categorical(label_encoder.transform(y_test1),num_classes=12)

In [ ]:
model_CNN = Sequential()
model_CNN.add(Conv1D(filters=32, kernel_size=3, activation='leaky_relu', input_shape=(tfidf_X_train.shape[1],1)))
model_CNN.add(MaxPooling1D(pool_size=3))
model_CNN.add(Flatten())
model_CNN.add(Dense(units=32, activation = 'leaky_relu'))
model_CNN.add(Dense(units = 16, activation = 'leaky_relu'))
model_CNN.add(Dropout(0.3))
model_CNN.add(Dense(units=12, activation='softmax'))

model_CNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_CNN.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 126, 32)             │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1344)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │          43,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 12)                  │             204 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 43,900 (171.48 KB)

 Trainable params: 43,900 (171.48 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Training

model_CNN.fit(tfidf_X_train, y_train1, epochs=50, validation_data=(tfidf_X_test, y_test1))

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7856 - loss: 0.4572 - val_accuracy: 0.4640 - val_loss: 1.3432
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7813 - loss: 0.4764 - val_accuracy: 0.4480 - val_loss: 1.3465
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8358 - loss: 0.4076 - val_accuracy: 0.4560 - val_loss: 1.3394
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7725 - loss: 0.4763 - val_accuracy: 0.4560 - val_loss: 1.3580
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7725 - loss: 0.4744 - val_accuracy: 0.4560 - val_loss: 1.3563
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8064 - loss: 0.4425 - val_accuracy: 0.4400 - val_loss: 1.3816
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.8061 - loss: 0.4178 - val_accuracy: 0.4640 - val_loss: 1.4045
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8344 - loss: 0.3872 - val_accuracy: 0.4400 - v

In [ ]:
from sklearn.metrics import classification_report

# Testing

y_pred_msg_CNN = model_CNN.predict(tfidf_X_test)
y_pred_classes = y_pred_msg_CNN.argmax(axis=1)
y_test_classes = (y_test1).argmax(axis=1)

print(classification_report(y_test_classes, y_pred_classes))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
              precision    recall  f1-score   support

           0       0.44      0.64      0.52        11
           1       0.53      0.82      0.64        11
           2       0.33      1.00      0.50         2
           3       0.50      0.36      0.42        11
           4       0.25      0.17      0.20         6
           5       0.33      0.20      0.25         5
           6       0.33      0.17      0.22        12
           7       0.56      0.69      0.62        13
           8       0.67      0.33      0.44        12
           9       0.48      0.61      0.54        18
          10       0.56      0.50      0.53        10
          11       0.27      0.21      0.24        14

    accuracy                           0.46       125
   macro avg       0.44      0.48      0.43       125
weighted avg       0.46      0.46      0.44       125



In [ ]:
# Finger probability array

from sklearn.naive_bayes import GaussianNB

y_train_class = np.argmax(y_train1,axis=1)
y_test_class = np.argmax(y_test1,axis=1)

gnb_fingers = GaussianNB()
gnb_fingers.fit(X_train1['fingers'].values.reshape(-1, 1),y_train_class)

y_pred_fingers = gnb_fingers.predict_proba(X_test1['fingers'].values.reshape(-1, 1))

In [ ]:
# Tail probability array

from sklearn.naive_bayes import MultinomialNB

label_encoder = LabelEncoder()
label_encoder.fit(df['tail'])

X_train_tail = (label_encoder.transform(X_train1['tail'])).reshape(-1,1)
X_test_tail = label_encoder.transform(X_test1['tail']).reshape(-1,1)

mnb_tail = MultinomialNB()
mnb_tail.fit(X_train_tail, y_train_class)

y_pred_tail = mnb_tail.predict_proba(X_test_tail)

y_pred_tail

array([[0.09866667, 0.11733333, 0.048     , ..., 0.112     , 0.09066667,
        0.09866667],
       [0.09866667, 0.11733333, 0.048     , ..., 0.112     , 0.09066667,
        0.09866667],
       [0.09866667, 0.11733333, 0.048     , ..., 0.112     , 0.09066667,
        0.09866667],
       ...,
       [0.09866667, 0.11733333, 0.048     , ..., 0.112     , 0.09066667,
        0.09866667],
       [0.09866667, 0.11733333, 0.048     , ..., 0.112     , 0.09066667,
        0.09866667],
       [0.09866667, 0.11733333, 0.048     , ..., 0.112     , 0.09066667,
        0.09866667]])

In [ ]:
y_pred_msg_log = np.log(y_pred_msg_CNN)
y_pred_fingers_log = np.log(y_pred_fingers)
y_pred_tail_log = np.log(y_pred_tail)

class_counts = df['species_group'].value_counts()
total_samples = len(df)
class_probabilities = np.array(class_counts / total_samples)

final_probabilities = y_pred_msg_log + y_pred_fingers_log + y_pred_tail_log - np.log(class_probabilities)
y_pred_final_classes = np.argmax(final_probabilities, axis=1)
print(classification_report(y_test_classes,y_pred_final_classes))

              precision    recall  f1-score   support

           0       0.47      0.64      0.54        11
           1       0.59      0.91      0.71        11
           2       1.00      1.00      1.00         2
           3       0.89      0.73      0.80        11
           4       1.00      0.33      0.50         6
           5       1.00      0.20      0.33         5
           6       0.43      0.25      0.32        12
           7       0.57      0.92      0.71        13
           8       0.83      0.42      0.56        12
           9       0.60      0.67      0.63        18
          10       0.67      0.80      0.73        10
          11       0.54      0.50      0.52        14

    accuracy                           0.62       125
   macro avg       0.72      0.61      0.61       125
weighted avg       0.66      0.62      0.60       125



## Ensemble Voting Model

In [ ]:
from scipy import sparse

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

class FinalLogClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.final_probabilities = None
        self.class_probabilities = None

    def fit(self, X, y, y_pred_msg_CNN, y_pred_fingers, y_pred_tail):
        y_pred_msg_log = np.log(y_pred_msg_CNN)
        y_pred_fingers_log = np.log(y_pred_fingers)
        y_pred_tail_log = np.log(y_pred_tail)

        class_counts = y.value_counts()
        total_samples = len(y)
        self.class_probabilities = np.array(class_counts / total_samples)

        self.final_probabilities = y_pred_msg_log + y_pred_fingers_log + y_pred_tail_log - np.log(self.class_probabilities)
        return self

    def predict(self, X):
        return np.argmax(self.final_probabilities, axis=1)

    def predict_proba(self, X):
        return np.exp(self.final_probabilities)

def preprocess_for_cnn(X):
    if sparse.issparse(X):
        X = X.toarray()
    return np.expand_dims(X, axis=-1)

def get_cnn_predictions(X):
    X_preprocessed = preprocess_for_cnn(X)
    return model_CNN.predict(X_preprocessed)

def combine_labels(y):
    return y.replace({
        'Faerix_Group1': 'Faerix',
        'Faerix_Group2': 'Faerix',
        'Emotivor_Group1': 'Emotivor',
        'Emotivor_Group2': 'Emotivor'
    })

class CustomRandomForestClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=300, max_depth=15, random_state=37, class_weight='balanced'):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.class_weight = class_weight
        self.rfc = RandomForestClassifier(n_estimators=self.n_estimators,
                                          max_depth=self.max_depth,
                                          random_state=self.random_state,
                                          class_weight=self.class_weight)

    def fit(self, X, y):
        self.rfc.fit(X, y)
        return self

    def predict(self, X):
        y_pred = self.rfc.predict(X)
        return combine_labels(pd.Series(y_pred))

    def predict_proba(self, X):
        return self.rfc.predict_proba(X)

    def get_params(self, deep=True):
        return {
            'n_estimators': self.n_estimators,
            'max_depth': self.max_depth,
            'random_state': self.random_state,
            'class_weight': self.class_weight
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        self.rfc = RandomForestClassifier(n_estimators=self.n_estimators,
                                          max_depth=self.max_depth,
                                          random_state=self.random_state,
                                          class_weight=self.class_weight)
        return self

class CustomLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, C=2, max_iter=1000, n_jobs=-1):
        self.C = C
        self.max_iter = max_iter
        self.n_jobs = n_jobs
        self.lr = LogisticRegression(C=self.C, max_iter=self.max_iter, n_jobs=self.n_jobs)

    def fit(self, X, y):
        self.lr.fit(X, y)
        return self

    def predict(self, X):
        y_pred = self.lr.predict(X)
        return combine_labels(pd.Series(y_pred))

    def predict_proba(self, X):
        return self.lr.predict_proba(X)

    def get_params(self, deep=True):
        return {
            'C': self.C,
            'max_iter': self.max_iter,
            'n_jobs': self.n_jobs
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        self.lr = LogisticRegression(C=self.C, max_iter=self.max_iter, n_jobs=self.n_jobs)
        return self

# Instantiate classifiers
custom_rfc = CustomRandomForestClassifier(n_estimators=300, max_depth=15, random_state=37, class_weight='balanced')
custom_lr = CustomLogisticRegression(C=2, max_iter=1000, n_jobs=-1)
mnb = MultinomialNB()
rfc = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=37)
svc = SVC(probability=True)
abc = AdaBoostClassifier(estimator=rfc, learning_rate=0.05, n_estimators=100)


# Calculate weights
total_f1 = 93.314794 + 84.582965 + 61.984111 + 87.189813 + 80.619854 + 58
weights = {
    'Logistic Regression': 102 / total_f1,
    'SVM': 84 / total_f1,
    'Naive Bayes': 69.33 / total_f1,
    'Random Forest': 95 / total_f1,
    'AdaBoost': 92 / total_f1,
    'final_log': 66 / total_f1
}

# Create the VotingClassifier
ec = VotingClassifier(
    estimators=[
        ('Multinomial NB', mnb),
        ('Custom Random Forest', custom_rfc),
        ('Custom Logistic Regression', custom_lr),
        ('Support Vector Machine', svc),
        ('Adaboost Random Forest', abc),
    ],
    voting='soft',
    weights=[weights['Naive Bayes'], weights['Random Forest'], weights['Logistic Regression'], weights['SVM'], weights['AdaBoost']]
)

# Fit the ensemble classifier
ec.fit(X_train, y_train)

# Predict and evaluate
y_pred = combined_predict(X_val)
print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [ ]:
Comparison_unibi = pd.DataFrame({'Logistic Regression': [accuracy_score(y_val,y_pred1)*100,f1_score(y_val,y_pred1,average='macro')*100,recall_score(y_val, y_pred1,average='micro')*100,precision_score(y_val, y_pred1,average='micro')*100],
                            'SVM':[accuracy_score(y_val,y_pred2)*100,f1_score(y_val,y_pred2,average='macro')*100,recall_score(y_val, y_pred2,average='micro')*100,precision_score(y_val, y_pred2,average='micro')*100],
                           'Naive Bayes':[accuracy_score(y_val,y_pred3)*100,f1_score(y_val,y_pred3,average='macro')*100,recall_score(y_val, y_pred3,average='micro')*100,precision_score(y_val, y_pred3,average='micro')*100],
                           'Random Forest':[accuracy_score(y_val,y_pred4)*100,f1_score(y_val,y_pred4,average='macro')*100,recall_score(y_val, y_pred4,average='micro')*100,precision_score(y_val, y_pred4,average='micro')*100],
                           'GradientBoosting':[accuracy_score(y_val,y_pred5)*100,f1_score(y_val,y_pred5,average='macro')*100,recall_score(y_val, y_pred5,average='micro')*100,precision_score(y_val, y_pred5,average='micro')*100],
                           'Ensembled':[accuracy_score(y_val,y_pred6)*100,f1_score(y_val,y_pred6,average='macro')*100,recall_score(y_val, y_pred6,average='micro')*100,precision_score(y_val, y_pred6,average='micro')*100],
                           'Adaboost':[accuracy_score(y_val,y_pred7)*100,f1_score(y_val,y_pred7,average='macro')*100,recall_score(y_val, y_pred7,average='micro')*100,precision_score(y_val, y_pred7,average='micro')*100],
})

In [ ]:
Comparison_unibi.rename(index={0:'Accuracy',1:'F1_score', 2: 'Recall',3:'Precision'}, inplace=True)
Comparison_unibi.head()

,Logistic Regression,SVM,Naive Bayes,Random Forest,GradientBoosting,Ensembled,Adaboost
Accuracy,92.000000,84.000000,69.333333,88.000000,76.000000,93.333333,84.000000
F1_score,93.314794,84.582965,61.984111,87.559222,74.700038,93.817137,82.425259
Recall,92.000000,84.000000,69.333333,88.000000,76.000000,93.333333,84.000000
Precision,92.000000,84.000000,69.333333,88.000000,76.000000,93.333333,84.000000
